In [13]:
# step 1 : import libraries and load the model 
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence 
from tensorflow.keras.models import load_model


In [14]:
# Load the imdb dataset word index

word_index=imdb.get_word_index()
reverse_word_index={value:key for key,value in word_index.items()}

In [15]:
# load the pretrained model
from tensorflow.keras.models import load_model

model = load_model('simple_rnn_imdb.h5')
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (32, 500, 128)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (32, 128)              │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 1)                │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [16]:
# step:2 helper function
# function to decode reviews

def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i-3,'?')for i in encoded_review])

In [17]:
# function to process the user input 

def preprocess_text(text):
    words=text.lower().split()
    encoded_review=[word_index.get(word,2)+3 for word in words]
    padded_review=sequence.pad_sequences([encoded_review],maxlen=500)
    padded_review

In [22]:
# predicting the function 

def predict_sentiment(review):
    preprocessed_input=preprocess_text(review)
    prediction=model.predict(preprocessed_input)
    sentiment='positive' if prediction[0][0] > 0.5  else 'negative'
    return sentiment,prediction[0][0]


In [23]:
# Step 1: Import libraries
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Step 2: Load dataset
max_features = 10000   # vocabulary size
maxlen = 200           # max length of review sequences
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

# Step 3: Build or Load Model
try:
    model = load_model("simple_mn_imdb.h5")
    print("Model loaded successfully!")
except:
    print("No saved model found. Training a new one...")
    model = Sequential()
    model.add(Embedding(max_features, 128, input_length=maxlen))
    model.add(LSTM(128))
    model.add(Dense(1, activation="sigmoid"))

    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    model.fit(x_train, y_train, epochs=1, batch_size=32, validation_split=0.2)

    model.save("simple_rnn_imdb.h5")

# Step 4: Tokenizer for custom reviews
word_index = imdb.get_word_index()
reverse_word_index = {value: key for (key, value) in word_index.items()}

tokenizer = Tokenizer(num_words=max_features)
tokenizer.word_index = word_index

# Step 5: Predict function
def predict_sentiment(review):
    seq = tokenizer.texts_to_sequences([review])
    padded = pad_sequences(seq, maxlen=maxlen)
    score = model.predict(padded)[0][0]
    sentiment = "Positive" if score >= 0.5 else "Negative"
    return sentiment, score

# Step 6: Test with example
example_review = "The movie was fantastic! I really loved it and would watch it again."
sentiment, score = predict_sentiment(example_review)

print("Review :", example_review)
print("Sentiment :", sentiment)
print("Prediction score :", score)


No saved model found. Training a new one...


c:\Users\VARADHAN\Desktop\DEEP LEARNING NLP\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


625/625 ━━━━━━━━━━━━━━━━━━━━ 66s 102ms/step - accuracy: 0.7595 - loss: 0.4892 - val_accuracy: 0.8182 - val_loss: 0.4241


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Review : The movie was fantastic! I really loved it and would watch it again.
Sentiment : Negative
Prediction score : 0.30765083
